In [1]:
import sys
sys.path.append("../")
from pyproj import CRS
import numpy as np
import sys
from osgeo import gdal
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from datetime import datetime
from time import time
from collections import Counter
import pandas as pd #for data analysing
import openpyxl
import os

In [3]:
DEM = xr.open_dataset('DEM_final.nc')

In [3]:
###j'ouvre dataset
NDSI_SON = xr.open_dataset('NDSI_DJF_step6_4nb_b_nc.nc')

In [4]:
####j'attribue un CRS à mon dataset
crs = CRS.from_string('EPSG:4326')
NDSI_SON.attrs["crs"] = crs.to_proj4()

/home/charlottepoussin/.conda/envs/myenv/lib/python3.12/site-packages/pyproj/crs/crs.py:1293: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [5]:
###j'ouvre le DEM
DEM100_a = xr.open_dataset('DEM100_b.nc')

In [6]:
DEM100_a = DEM100_a.isel(z=0)

In [7]:
DEM100_a.attrs["crs"] = crs.to_proj4()

In [8]:
###on veut metre le DEM dans le dataset
lat = NDSI_SON['latitude'].values
lon = NDSI_SON['longitude'].values
time = NDSI_SON['z'].values

dem = xr.DataArray(DEM100_a.layer.values, coords={'latitude': lat,'longitude':lon}, dims=['latitude','longitude'])

In [9]:
NDSI_SON['DEM'] = dem

In [10]:
####on change en entier le DEM on a pas besoisn de virgule
NDSI_SON['DEM'] = NDSI_SON.DEM.astype('int16')

/home/charlottepoussin/.conda/envs/myenv/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:213: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [11]:
##les valeur hors suisse son en 0 et non en NAN por le DEM on va alors les changer en NAN
NDSI_SON['DEM'] = NDSI_SON.DEM.where(NDSI_SON.DEM != 0)

In [13]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_cloudpercent_b(array):
    cloud = np.sum(array == 2)
    #print(cloud)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    percentage_cloud_b = round(((float(cloud) / float(tot_pix)) * 100),2)
    return percentage_cloud_b

In [14]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_snowpercent_b(array):
    snow = np.count_nonzero((array == 1) | (array == 100))
    #print(snow)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    snow_percent_b = round(((float(snow) / float(tot_pix)) * 100),2)
    return snow_percent_b

In [15]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_landpercent_b(array):
    land = np.count_nonzero((array == 0) | (array == 200))
    #print(land)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    land_percent_b = round((float(land) / float(tot_pix)) * 100,2)
    return land_percent_b

In [16]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_cloudpercent_a(array_copy):
    cloud = np.sum(array_copy  == 2)
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    cloud_percent = round((float(cloud) / float(tot_pix)) * 100, 2)
    return cloud_percent

In [17]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_snowpercent_a(array_copy):
    snow = np.count_nonzero((array_copy == 1) | (array_copy == 100))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    snow_percent = round((float(snow) / float(tot_pix)) * 100, 2)
    return snow_percent

In [18]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_landpercent_a(array_copy):
    land = np.count_nonzero((array_copy == 0) | (array_copy == 200))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    land_percent = round((float(land) / float(tot_pix)) * 100, 2)
    return land_percent

In [19]:
def step5_gapfilling_cloud(array,array_copy):
    cloud_b = np.sum(array==2)
    cloud_a = np.sum(array_copy==2)
    cloud_step5 = cloud_b - cloud_a 
    step5_gf_cloud = round((float(cloud_step5)/float(cloud_b))*100,2)
    #step3_gf_cloud = ((cloud_step3)/(cloud_b))*100
    return step5_gf_cloud

In [20]:
def step5_gapfilling_tot(array,array_copy):
    cloudpercent_b = compute_cloudpercent_b(array)
    cloudpercent_a = compute_cloudpercent_a(array_copy)
    step5_gf_tot = round(((cloudpercent_b)-(cloudpercent_a)),2)
    #step3_gf_tot = ((cloudpercent_b)-(cloudpercent_a))
    return step5_gf_tot

In [21]:
def step5_internal(array, array_dem):

    # Je crée une copie pour pouvoir travailler dessus sans changer l'initial
    snow_step5 = array.copy(deep=True)

    while True:
        # Get 8 neighboring pixels for raster and dem
        W = snow_step5.values[1:-1, :-2]
        NW = snow_step5.values[:-2, :-2]
        N = snow_step5.values[:-2, 1:-1]
        NE = snow_step5.values[:-2, 2:]
        E = snow_step5.values[1:-1, 2:]
        SE = snow_step5.values[2:, 2:]
        S = snow_step5.values[2:, 1:-1]
        SW = snow_step5.values[2:, :-2]

        Wdem = array_dem.values[1:-1, :-2]
        NWdem = array_dem.values[:-2, :-2]
        Ndem = array_dem.values[:-2, 1:-1]
        NEdem = array_dem.values[:-2, 2:]
        Edem = array_dem.values[1:-1, 2:]
        SEdem = array_dem.values[2:, 2:]
        Sdem = array_dem.values[2:, 1:-1]
        SWdem = array_dem.values[2:, :-2]

        arrdem = array_dem.values[1:-1, 1:-1]
    
        mask_snow = (
        ((((W == 1)| (W == 100)) & (
            arrdem > Wdem)) | (
                ((N == 1)| (N == 100)) & (
                     arrdem > Ndem)) | (
                        ((E == 1)| (E == 100)) & (
                        arrdem > Edem)) | (
                            ((S == 1)| (S == 100)) & (
                                arrdem > Sdem)) | (
                                    ((NW == 1)| (NW == 100)) & (
                                        arrdem > NWdem)) | (
                                            ((NE == 1)| (NE == 100)) & (
                                                arrdem > NEdem)) | (
                                                    ((SE == 1)| (SE == 100)) & (
                                                        arrdem > SEdem)) | (
                                                            ((SW == 1)| (SW == 100)) & (
                                                                arrdem > SWdem))) & (
                                                                    snow_step5.values[
                                                                        1:-1, 1:-1] == 2))

        # Use the mask to set corresponding elements
        snow_step5.values[1:-1, 1:-1][mask_snow] = 100

            # Check if any modification was made in the current iteration
        if not mask_snow.any():
            break  # Exit the loop if no modifications were made

    return snow_step5

In [22]:
def step5(array, array_dem):
    # four-pixels neighboring
    print("Starting step 5")
    ##Calculer le % de nuage avant le processus
    percentage_cloud_b = compute_cloudpercent_b(array)
    print("cloud percent before step 5 : " + str(round(percentage_cloud_b,2)))
    percentage_snow_b = compute_snowpercent_b(array)
    percentage_land_b = compute_landpercent_b(array)
    
    
    # compute 4 pixel snow neighboring
    snow_step5 = step5_internal(array,array_dem)

    percentage_cloud_a = compute_cloudpercent_a(snow_step5)
    print("cloud percent after step 5 : " + str(round(percentage_cloud_a,2)))
    percentage_snow_a = compute_snowpercent_a(snow_step5)
    percentage_land_a = compute_landpercent_a(snow_step5)

    step5_gf_tot = step5_gapfilling_tot(array,snow_step5)
    print("diminution of cloud tot (%) : " + str(round(step5_gf_tot,2)))
    step5_gf_cloud = step5_gapfilling_cloud(array,snow_step5)
    print("diminution of cloud (%) : " + str(round(step5_gf_cloud,2)))
    

    print("End of step 5")

    return snow_step5


In [23]:
cloud_percent_b_list = []
snow_percent_b_list = []
land_percent_b_list = []
cloud_percent_a_list = []
snow_percent_a_list = []
land_percent_a_list = []
step5_gapfilling_tot_list = []
step5_gapfilling_cloud_list = []
time_list = []
snow_step5_list = []

In [24]:
for i in range(len(NDSI_SON.z)):
    print(i)
    snow_step5 = step5(NDSI_SON.snow_step4.isel(z=i),NDSI_SON.DEM)
    snow_step5_list.append(snow_step5)
    cloud_percent_b = compute_cloudpercent_b(NDSI_SON.snow_step4.isel(z=i))
    cloud_percent_b_list.append(cloud_percent_b)
    snow_percent_b = compute_snowpercent_b(NDSI_SON.snow_step4.isel(z=i))
    snow_percent_b_list.append(snow_percent_b)
    land_percent_b = compute_landpercent_b(NDSI_SON.snow_step4.isel(z=i))
    land_percent_b_list.append(land_percent_b)
    time = pd.to_datetime(NDSI_SON.z.isel(z=i).values)
    formatted_date = time.strftime('%Y-%m-%d')
    time_list.append(formatted_date)

    cloud_percent_a = compute_cloudpercent_a(snow_step5)
    cloud_percent_a_list.append(cloud_percent_a)
    snow_percent_a = compute_snowpercent_a(snow_step5)
    snow_percent_a_list.append(snow_percent_a)
    land_percent_a = compute_landpercent_a(snow_step5)
    land_percent_a_list.append(land_percent_a)

    step5_gapfilling_total = step5_gapfilling_tot(NDSI_SON.snow_step4.isel(z=i),snow_step5)
    step5_gapfilling_tot_list.append(step5_gapfilling_total)
    step5_gapfilling_clouds = step5_gapfilling_cloud(NDSI_SON.snow_step4.isel(z=i),snow_step5)
    step5_gapfilling_cloud_list.append(step5_gapfilling_clouds)


0
Starting step 5
cloud percent before step 5 : 44.7
cloud percent after step 5 : 44.12
diminution of cloud tot (%) : 0.58
diminution of cloud (%) : 1.29
End of step 5
1
Starting step 5
cloud percent before step 5 : 24.53
cloud percent after step 5 : 24.43
diminution of cloud tot (%) : 0.1
diminution of cloud (%) : 0.41
End of step 5
2
Starting step 5
cloud percent before step 5 : 49.18
cloud percent after step 5 : 48.75
diminution of cloud tot (%) : 0.43
diminution of cloud (%) : 0.87
End of step 5
3
Starting step 5
cloud percent before step 5 : 63.26
cloud percent after step 5 : 62.69
diminution of cloud tot (%) : 0.57
diminution of cloud (%) : 0.91
End of step 5
4
Starting step 5
cloud percent before step 5 : 3.26
cloud percent after step 5 : 3.14
diminution of cloud tot (%) : 0.12
diminution of cloud (%) : 3.5
End of step 5
5
Starting step 5
cloud percent before step 5 : 70.73
cloud percent after step 5 : 70.12
diminution of cloud tot (%) : 0.61
diminution of cloud (%) : 0.86
End o

In [25]:
snow_step5_concat = xr.concat(snow_step5_list, dim='z')
dataset_step5 = xr.Dataset({'snow_step5': snow_step5_concat})

In [26]:
data = {
    'Time': time_list,
    'Cloud Percent Before': cloud_percent_b_list,
    'Snow Percent Before': snow_percent_b_list,
    'Land Percent Before': land_percent_b_list,
    'Cloud Percent After': cloud_percent_a_list,
    'Snow Percent After': snow_percent_a_list,
    'Land Percent After': land_percent_a_list,
    'Cloud Reduction (%)': step5_gapfilling_tot_list,
    'Cloud Reduction After (%)': step5_gapfilling_cloud_list,
}

# Créer un DataFrame à partir du dictionnaire
df_step5 = pd.DataFrame(data)
filename = f'df_step7_8nb_DJF_b.xlsx'
filepath = os.path.join('/home/charlottepoussin/Excels', filename)
df_step5.to_excel(filepath, index=False)